In [102]:
!pip install nb_black -q
!pip install pdfminer.six -q
!pip install urllib3 -q
!pip install bs4 -q

In [103]:
%load_ext lab_black

### Aluno Marco Antonio Moreira Carujo

# Atividade 1: Coleta de Documentos em PDF
Atividade: Baixar arquivos PDFs da Web e armazená-los como arquivo de texto.

Site: 2019 - Anais do IV Congresso sobre Tecnologias na Educação (https://sol.sbc.org.br/index.php/ctrle)

In [118]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from joblib import Parallel, delayed

Dentro do site temos dois links um comprido onde mostra apenas os ultimos artigos e um completo onde temos todos os artigos (75 artigos) do ano de 2019.

Podemos baixar ele e ir na tag **\<a>** buscar o href de cada um. Dentro do href apenas mudando a parametro/rota view para download podemos baixar cada um dos PDF's.
    
A tag a possuí uma classe chama **obj_galley_link pdf** o que facilita o encontro da mesma.

In [110]:
response = urlopen("https://sol.sbc.org.br/index.php/ctrle")  # Comprimido
response = urlopen("https://sol.sbc.org.br/index.php/ctrle/issue/view/486")  # Completo
html = response.read()

soup = BeautifulSoup(html, "html.parser")
elements = soup.find_all("a", class_="obj_galley_link pdf")
print("Quantos artigos temos no site?", len(elements), "artigos.")

Quantos artigos temos no site? 75 artigos.


Apenas manipulações de objetos e tranformações de Strings para gerar as seguintes listas:
- lista de links de view
- lista de links de download
- lista de links de nomes de arquivos para ser tanto .pdf quanto .txt

In [111]:
links_to_view = [el["href"] for el in elements]
links_to_view[:5]

['https://sol.sbc.org.br/index.php/ctrle/article/view/8870/8771',
 'https://sol.sbc.org.br/index.php/ctrle/article/view/8871/8772',
 'https://sol.sbc.org.br/index.php/ctrle/article/view/8872/8773',
 'https://sol.sbc.org.br/index.php/ctrle/article/view/8873/8774',
 'https://sol.sbc.org.br/index.php/ctrle/article/view/8874/8775']

In [112]:
links_to_download = [el.replace("view", "download") for el in links_to_view]
links_to_download[:5]

['https://sol.sbc.org.br/index.php/ctrle/article/download/8870/8771',
 'https://sol.sbc.org.br/index.php/ctrle/article/download/8871/8772',
 'https://sol.sbc.org.br/index.php/ctrle/article/download/8872/8773',
 'https://sol.sbc.org.br/index.php/ctrle/article/download/8873/8774',
 'https://sol.sbc.org.br/index.php/ctrle/article/download/8874/8775']

In [113]:
names_to_download = [el[-9:].replace("/", "-") for el in links_to_download]
names_to_download[:5]

['8870-8771', '8871-8772', '8872-8773', '8873-8774', '8874-8775']

As função **download** e **transforma em texto** foram retiradas/inspiradas no material da aula.

A função worker tem como objetivo encapsular a tarefa de baixar e transformar para ser paraleliada.

In [117]:
import requests
from pdfminer.high_level import extract_text
import os


def download(url, nome):
    resposta = requests.get(url)
    if resposta.status_code == 200:
        with open(os.path.join("pdf", nome + ".pdf"), "wb") as f:
            f.write(resposta.content)


def transforma_em_texto(nome):
    pdf_text = extract_text(os.path.join("pdf", nome + ".pdf"), codec="utf-8")
    with open(os.path.join("text", nome + ".txt"), "w", encoding="utf-8") as f:
        f.write(pdf_text)
        
def worker(i):
    link = links_to_download[i]
    name = names_to_download[i]
    download(link, name)
    transforma_em_texto(name)
    return f'Arquivo {name} pdf e txt gerado e salvo!'

Execução da celula seguir irá baixar paralelamente 75 artigos e transformar em texto. Os PDFs serão salvos no diretorio *pdf* e os arquivos txt no diretorio *txt*.

In [121]:
%%time
os.makedirs("pdf", exist_ok=True)
os.makedirs("text", exist_ok=True)

Parallel(verbose=1, n_jobs=-1)(delayed(worker)(i) for i in range(len(names_to_download)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    8.2s


Wall time: 19.2 s


[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:   19.1s finished


['Arquivo 8870-8771 pdf e txt gerado e salvo!',
 'Arquivo 8871-8772 pdf e txt gerado e salvo!',
 'Arquivo 8872-8773 pdf e txt gerado e salvo!',
 'Arquivo 8873-8774 pdf e txt gerado e salvo!',
 'Arquivo 8874-8775 pdf e txt gerado e salvo!',
 'Arquivo 8875-8776 pdf e txt gerado e salvo!',
 'Arquivo 8876-8777 pdf e txt gerado e salvo!',
 'Arquivo 8877-8778 pdf e txt gerado e salvo!',
 'Arquivo 8878-8779 pdf e txt gerado e salvo!',
 'Arquivo 8879-8780 pdf e txt gerado e salvo!',
 'Arquivo 8880-8781 pdf e txt gerado e salvo!',
 'Arquivo 8881-8782 pdf e txt gerado e salvo!',
 'Arquivo 8882-8783 pdf e txt gerado e salvo!',
 'Arquivo 8883-8784 pdf e txt gerado e salvo!',
 'Arquivo 8884-8785 pdf e txt gerado e salvo!',
 'Arquivo 8885-8786 pdf e txt gerado e salvo!',
 'Arquivo 8886-8787 pdf e txt gerado e salvo!',
 'Arquivo 8887-8788 pdf e txt gerado e salvo!',
 'Arquivo 8888-8789 pdf e txt gerado e salvo!',
 'Arquivo 8889-8790 pdf e txt gerado e salvo!',
 'Arquivo 8890-8791 pdf e txt gerado e s